In [1]:
# Pull Sportsnet Player Lists
import urllib3
import certifi
from bs4 import BeautifulSoup
import pandas as pd

http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())

base_url = "https://fantasy.sportsnet.ca/sportsnet/baseball18/player_stats"
group_param = "pick_stats_group"

scrape_pages = {
    "C/DH": 1,
    "1B/3B": 2,
    "2B/SS": 3,
    "OF": 4,
    "SP": 5,
    "RP": 6
}

frames = {}

for key, value in scrape_pages.items():

    response = http.request("GET", base_url + "?" + group_param + "=" + str(value))
    
    soup = BeautifulSoup(response.data, 'html.parser')
    table = soup.find('table', attrs={'id': 'playerstats'})
    header = [th.text for th in table.find('thead').select('th')]
    players = [[td.text.strip().split('\n')[0] for td in row.select('td')] for row in table.tbody.find_all('tr')]
    cols = zip(*players)
    tbl_d = {name: col for name, col in zip(header, cols)}
    df = pd.DataFrame(tbl_d, columns=header)
    name = df['Player'].str.split(', ')
    df["First"] = name.str[1]
    df['Last'] = name.str[0]
    df['Player'] = name.str[1] + " " + name.str[0]
    df.apply(pd.to_numeric, errors='ignore')
    frames[key] = df

In [2]:
# Load Fantasy Pro projections
hitters = pd.read_csv("data/FantasyPros_2018_wk2_Projections_H.csv")
pitchers = pd.read_csv("data/FantasyPros_2018_wk2_Projections_P.csv")

In [3]:
# Join Sportsnet data with projects
for key, value in frames.items():
    merge_frame = hitters
    if key == "SP" or key == "RP":
        merge_frame = pitchers
    result = pd.merge(value, merge_frame, on='Player')
    frames[key] = result

In [4]:
# Build all possible Teams into a dataframe
teams = pd.DataFrame(columns=["C/DH", "1B/3B", "2B/SS", "OF1", "OF2", "SP", "RP", "PV Total", "xScore"])
i = 0

for cdh_index, cdh in frames["C/DH"].iterrows():
    for fbtb_index, fbtb in frames["1B/3B"].iterrows():
        for sbss_index, sbss in frames["2B/SS"].iterrows():
            for of1_index, of1 in frames["OF"].iterrows():
                for of2_index, of2 in frames["OF"].iterrows():
                    if of1["Player"] == of2["Player"]:
                        continue
                    if of2['PV'] + of1['PV'] + sbss['PV'] + fbtb['PV'] + cdh['PV'] >= 20:
                        continue
                    for sp_index, sp in frames["SP"].iterrows():
                        if sp['PV'] + of2['PV'] + of1['PV'] + sbss['PV'] + fbtb['PV'] + cdh['PV'] >= 20:
                            continue
                        for rp_index, sp in frames["RP"].iterrows():
                            if rp['PV'] + sp['PV'] + of2['PV'] + of1['PV'] + sbss['PV'] + fbtb['PV'] + cdh['PV'] >= 20:
                                continue
                            i = i + 1
print(i)

TypeError: '>=' not supported between instances of 'str' and 'int'